# QIIME 2 Tutorial 
### QIIME 2 is a powerful platform for microbial ecology analysis.

### The use of QIIME 2 allows for detailed analysis of microbial communities, making it an essential tool for researchers studying microbial dynamics in clinical or environmental samples

### The tutorial is structured to be completed within a Jupyter Notebook, offering an interactive and flexible platform for conducting bioinformatics analyses.

### By following this tutorial, participants will gain hands-on experience with key techniques such as denoising sequencing data, performing diversity analyses, and conducting taxonomic classifications.

### By the end of the tutorial, participants will be familiar with the core steps of microbial data analysis, providing them with insights into the composition and diversity of microbiota, similar to those studied in the paper "The cervical microbiota of Hispanics living in Puerto Rico is nonoptimal regardless of HPV status" 

https://journals.asm.org/doi/10.1128/msystems.00357-23#:~:text=However%2C%20no%20significant%20associations%20between,of%20a%20woman's%20physiological%20stage.

### 1. Import the Required QIIME 2 Libraries

In [9]:
import os
import qiime2
from qiime2.plugins import dada2, feature_table, phylogeny, diversity, taxa, metadata

# Useful for viewing visualizations directly in the notebook
from qiime2 import Visualization

In [8]:
!wget -O demux.qza https://docs.qiime2.org/2024.5/data/tutorials/moving-pictures/demux.qza

--2024-09-17 11:31:43--  https://docs.qiime2.org/2024.5/data/tutorials/moving-pictures/demux.qza
Resolving docs.qiime2.org (docs.qiime2.org)... 172.67.186.144, 104.21.84.49
Connecting to docs.qiime2.org (docs.qiime2.org)|172.67.186.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20604579 (20M) [application/octet-stream]
Saving to: ‘demux.qza’

demux.qza            47%[========>           ]   9.28M  10.1KB/s    in 7m 52s  

2024-09-17 11:39:40 (20.2 KB/s) - Read error at byte 9731802/20604579 (Connection reset by peer). Retrying.

--2024-09-17 11:39:41--  (try: 2)  https://docs.qiime2.org/2024.5/data/tutorials/moving-pictures/demux.qza
Connecting to docs.qiime2.org (docs.qiime2.org)|172.67.186.144|:443... connected.
Unable to establish SSL connection.


### 2. Load Data

#### Load your data (the demultiplexed sequences and metadata file) into QIIME 2 artifacts


In [18]:
!wget -O sample-metadata.tsv https://data.qiime2.org/2024.5/tutorials/moving-pictures/sample_metadata.tsv
!mkdir -p emp-single-end-sequences
!wget -O emp-single-end-sequences/barcodes.fastq.gz https://data.qiime2.org/2024.5/tutorials/moving-pictures/emp-single-end-sequences/barcodes.fastq.gz
!wget -O emp-single-end-sequences/sequences.fastq.gz https://data.qiime2.org/2024.5/tutorials/moving-pictures/emp-single-end-sequences/sequences.fastq.gz


--2024-09-17 12:40:34--  https://data.qiime2.org/2024.5/tutorials/moving-pictures/sample_metadata.tsv
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://docs.google.com/spreadsheets/d/e/2PACX-1vTKz0BaPz69jai6JJEr04Wg9CNUHqXqwMINldi0CFCz8_IUWu9MOwRBErJ7ljMaP86vEa1rZ3AyKl_K/pub?gid=0&single=true&output=tsv [following]
--2024-09-17 12:40:34--  https://docs.google.com/spreadsheets/d/e/2PACX-1vTKz0BaPz69jai6JJEr04Wg9CNUHqXqwMINldi0CFCz8_IUWu9MOwRBErJ7ljMaP86vEa1rZ3AyKl_K/pub?gid=0&single=true&output=tsv
Resolving docs.google.com (docs.google.com)... 192.178.50.78
Connecting to docs.google.com (docs.google.com)|192.178.50.78|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-0o-60-sheets.googleusercontent.com/pub/54bogvaave6cua4cdnls17ksc4/7a70vupd17vsgl1no6jjku5gn4/1726590765000/1052505

In [19]:
!qiime tools import \
  --type EMPSingleEndSequences \
  --input-path emp-single-end-sequences \
  --output-path emp-single-end-sequences.qza


Imported emp-single-end-sequences as EMPSingleEndDirFmt to emp-single-end-sequences.qza


In [20]:
!qiime demux emp-single \
  --i-seqs emp-single-end-sequences.qza \
  --m-barcodes-file sample-metadata.tsv \
  --m-barcodes-column barcode-sequence \
  --o-per-sample-sequences demux.qza \
  --o-error-correction-details demux-details.qza


Saved SampleData[SequencesWithQuality] to: demux.qza
Saved ErrorCorrectionDetails to: demux-details.qza


In [21]:
!qiime demux summarize \
  --i-data demux.qza \
  --o-visualization demux.qzv


Saved Visualization to: demux.qzv


In [22]:
!qiime tools view demux.qzv


Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

In [25]:
import qiime2
from qiime2.plugins import demux, feature_table, diversity, taxa, metadata

# Display visualizations
from qiime2 import Visualization


In [26]:
demuxed_seqs = qiime2.Artifact.load('demux.qza')
metadata_file = qiime2.Metadata.load('sample-metadata.tsv')


In [27]:
demux_summary = demux.visualizers.summarize(demuxed_seqs)
demux_summary.visualization


/opt/homebrew/Caskroom/miniforge/base/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/opt/homebrew/Caskroom/miniforge/base/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_demux/_summarize/_visualizer.py:180: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  context['result_data'] = pd.concat([context['result_data'], df])
/opt/homebrew/Caskroom/miniforge/base/envs/qiime2-amplicon-2024.5/lib/python3.9/site-packages/q2_demux/_summarize/_visualizer.py:74: FutureWarning: Setting an item of incompatible dtype is

<visualization: Visualization uuid: 4b6bb9a6-a5f9-4740-a258-83d27d97c3c8>

In [30]:
demux_summary.visualization.save('demux_summary.qzv')


'demux_summary.qzv'